In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#reading csv file into dataframe for hospitals
csv_file = 'Resources/Hospitals.csv'
hospital_data_df = pd.read_csv(csv_file)
hospital_data_df.head()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,2017-12-18T00:00:00.000Z,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE


In [3]:
#reading csv file into dataframe for income
csv_file = 'Resources/income.csv'
income_data_df = pd.read_csv(csv_file)
income_data_df.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [6]:
#Check to see if there are duplicates within the dataframe
dup_hospital_data = hospital_data_df.duplicated()
dup_hospital_data

0       False
1       False
2       False
3       False
4       False
        ...  
7565    False
7566    False
7567    False
7568    False
7569    False
Length: 7570, dtype: bool

In [7]:
dup_income_data = income_data_df.duplicated()
dup_income_data

0        False
1        False
2        False
3        False
4        False
         ...  
32521    False
32522    False
32523    False
32524    False
32525    False
Length: 32526, dtype: bool

In [8]:
#Check if zip codes in income dataset are unique. This will enable a join in SQL
income_data_df['Zip_Code'].value_counts()

94606    19
11203    16
11377    15
11204    15
92626    15
         ..
6820      1
2726      1
53933     1
15024     1
36862     1
Name: Zip_Code, Length: 14803, dtype: int64

In [9]:
#creating sub df for just the columns we want. Removed id columns.
hospital_cols = ["ZIP", "STATE", "CITY", "POPULATION", "BEDS", "NAME", "OWNER", "STATUS", "TYPE"]
hospital_transformed = hospital_data_df[hospital_cols].copy()


income_cols = ["Zip_Code", "Mean", "Median", "Stdev", "sum_w"]
income_transformed = income_data_df[income_cols].copy()

In [10]:
#rename the column headers
hospital_renamed = hospital_transformed.rename(columns= { 
                                                             "ZIP":"Zip", 
                                                             "STATE":"Us_State", 
                                                             "CITY":"City", 
                                                             "POPULATION":"Pop_100k", 
                                                             "BEDS":"Beds", 
                                                             "NAME":"Hospital_Name", 
                                                             "OWNER":"Type_Owner", 
                                                             "STATUS":"Status", 
                                                             "TYPE":"Care"})
income_renamed = income_transformed.rename(columns= {"Zip_Code":"Zip",
                                                         "sum_w":"Households"})

In [11]:
hospital_renamed.head()

,Zip,Us_State,City,Pop_100k,Beds,Hospital_Name,Type_Owner,Status,Care
0,77520,TX,BAYTOWN,182,182.0,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,NON-PROFIT,OPEN,GENERAL ACUTE CARE
1,43230,OH,COLUMBUS,50,50.0,"WOODS AT PARKSIDE,THE",PROPRIETARY,OPEN,SPECIAL
2,45404,OH,DAYTON,155,155.0,DAYTON CHILDREN'S HOSPITAL,NON-PROFIT,OPEN,CHILDREN
3,44512,OH,BOARDMAN,45,45.0,VIBRA HOSPITAL OF MAHONING VALLEY,PROPRIETARY,OPEN,LONG TERM CARE
4,45417,OH,DAYTON,32,32.0,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,PROPRIETARY,OPEN,PSYCHIATRIC


In [12]:
income_renamed.head()

,Zip,Mean,Median,Stdev,Households
0,36611,38773,30506,33101,1638.260513
1,36048,37725,19528,43789,258.017685
2,35051,54606,31930,57348,926.031000
3,36572,63919,52814,47707,378.114619
4,36528,77948,67225,54270,282.320328


In [22]:
#Group by zip, then take average across all the fields in the income data
income_zip_grp = income_renamed.groupby('Zip').mean(['Mean', 'Median','Stdev', 'Households'])
income_zip_grp

,Mean,Median,Stdev,Households
Zip,,,,
601,22496.333333,13144.333333,27148.666667,794.505347
602,15078.400000,11323.600000,12503.600000,254.797362
603,26046.800000,15386.400000,29969.600000,491.652170
605,27497.500000,9461.000000,41453.500000,386.318115
606,19194.000000,15233.000000,15261.000000,558.299922
...,...,...,...,...
99835,80041.000000,59519.000000,65235.000000,134.493046
99840,81352.000000,69886.000000,49197.000000,174.689074
99901,57572.666667,50320.666667,39288.666667,300.340333


In [23]:
#hospital_transformed.dtypes

In [24]:
#income_transformed['id'] = income_transformed['id'].astype(int)

In [25]:
df_merged = hospital_renamed.merge(income_zip_grp,how='left',left_on='Zip',right_on='Zip')


In [26]:
#df_merged = hospital_transformed.merge(income_transformed,how='left',left_on='Hospital_id',right_on='id')

In [27]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7570 entries, 0 to 7569
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Zip            7570 non-null   int64  
 1   Us_State       7570 non-null   object 
 2   City           7570 non-null   object 
 3   Pop_100k       7570 non-null   int64  
 4   Beds           7569 non-null   float64
 5   Hospital_Name  7570 non-null   object 
 6   Type_Owner     7569 non-null   object 
 7   Status         7570 non-null   object 
 8   Care           7570 non-null   object 
 9   Mean           6176 non-null   float64
 10  Median         6176 non-null   float64
 11  Stdev          6176 non-null   float64
 12  Households     6176 non-null   float64
dtypes: float64(5), int64(2), object(6)
memory usage: 828.0+ KB


In [ ]:
#income_transformed.Zip.value_counts()

In [ ]:
#income_transformed[income_transformed.Zip == 94606]

In [17]:
#creating connection to sql 
conn_str = 'postgres:postgres@localhost:5432/Income_vs_healthcare'
engine = create_engine(f'postgresql://{conn_str}')

In [28]:
#check connection
engine.table_names()

<ipython-input-28-64323e9d97e4>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['healthcare', 'income']

In [33]:
#loading cleaned data into sql db healthcare table
hospital_renamed.to_sql(name='healthcare',con=engine,if_exists='append',index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "healthcare_pkey"
DETAIL:  Key ("Zip")=(58701) already exists.

[SQL: INSERT INTO healthcare ("Zip", "Us_State", "City", "Pop_100k", "Beds", "Hospital_Name", "Type_Owner", "Status", "Care") VALUES (%(Zip)s, %(Us_State)s, %(City)s, %(Pop_100k)s, %(Beds)s, %(Hospital_Name)s, %(Type_Owner)s, %(Status)s, %(Care)s)]
[parameters: ({'Zip': 77520, 'Us_State': 'TX', 'City': 'BAYTOWN', 'Pop_100k': 182, 'Beds': 182.0, 'Hospital_Name': 'HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXANDER CAMPUS', 'Type_Owner': 'NON-PROFIT', 'Status': 'OPEN', 'Care': 'GENERAL ACUTE CARE'}, {'Zip': 43230, 'Us_State': 'OH', 'City': 'COLUMBUS', 'Pop_100k': 50, 'Beds': 50.0, 'Hospital_Name': 'WOODS AT PARKSIDE,THE', 'Type_Owner': 'PROPRIETARY', 'Status': 'OPEN', 'Care': 'SPECIAL'}, {'Zip': 45404, 'Us_State': 'OH', 'City': 'DAYTON', 'Pop_100k': 155, 'Beds': 155.0, 'Hospital_Name': "DAYTON CHILDREN'S HOSPITAL", 'Type_Owner': 'NO

In [34]:
#loading cleaned data into sql db income table
income_zip_grp.to_sql(name='income',con=engine,if_exists='append',index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "Zip" violates not-null constraint
DETAIL:  Failing row contains (null, 22496, 13144, 795, 27149).

[SQL: INSERT INTO income ("Mean", "Median", "Stdev", "Households") VALUES (%(Mean)s, %(Median)s, %(Stdev)s, %(Households)s)]
[parameters: ({'Mean': 22496.333333333332, 'Median': 13144.333333333334, 'Stdev': 27148.666666666668, 'Households': 794.5053474052617}, {'Mean': 15078.4, 'Median': 11323.6, 'Stdev': 12503.6, 'Households': 254.79736170099423}, {'Mean': 26046.8, 'Median': 15386.4, 'Stdev': 29969.6, 'Households': 491.6521697370472}, {'Mean': 27497.5, 'Median': 9461.0, 'Stdev': 41453.5, 'Households': 386.3181154836765}, {'Mean': 19194.0, 'Median': 15233.0, 'Stdev': 15261.0, 'Households': 558.299922171477}, {'Mean': 26944.333333333332, 'Median': 19881.666666666668, 'Stdev': 23642.333333333332, 'Households': 673.9501484790156}, {'Mean': 26967.0, 'Median': 18853.5, 'Stdev': 26828.0, 'Households': 600.0108960328818}, {

In [35]:
#check hospital data
pd.read_sql_query('select * from healthcare', con=engine).head()

,Zip,Us_State,City,Pop_100k,Beds,Hospital_Name,Type_Owner,Status,Care


In [36]:
#check income data
pd.read_sql_query('select * from income', con=engine).head()

,Zip,Mean,Median,Households,Stdev
